# Current to do's 2022-28

- naming of coordinates - x horizontal, y vertical - encapsulate the differences in dic_grid, so that any further level works 
   without swapping
 - extend interpolation in dsf to cdf - including the time index - this will allow for the crack detection and spline representation    of the functions 
$$
\omega_C( t, y )
$$
 - use this function to determine
$$
y^\mathrm{tip}_C( t )
$$ 
$$
\Delta u_{Ca}( y, t ) = u( x+r, y, t ) - u( x-r, y, t ) 
$$
 - Constitutive laws applied across the crack ligament
$$
 \sigma_{Cab}(y, t) = \mathcal{S}_{ab}(\Delta u{Ca}(y, t))
$$
 - distinguish the change events - finer resolution of the tag types (TIME, ALG, MESH).

# Crack detection strategy

The detection algorithm uses the knowledge about shear crack propagation 
 - distance between cracks is related to the shielded length given by the reinforcement ratio      and bond properties
 - cracks start from the bottom of the cross section
 - cracks propagate upwards and bend in the direction of an increasing cross-sectional moment
 
The crack pattern in the shear zone gets most visible prior shortly before the maximum load has been reached. Near to the failure, the regularity of the crack pattern is lost and the specimen cannot be viewed as a series of individual shear cracks any more. The initiation of failure via a horizontal crossing crack initiated by a vertical stress component is initiated either near the reinforcement layer due to the dowel action or by a crack tip propagating horizontally through a compression zone.

## Full crack detection strategy

This observation is exploited by the crack detection algorithm by adapting the following strategy. 

 - The displacement field $U_{IJa}^{F_\gamma}$ corresponding to $F_{\gamma} = 0.9 F_{\max}$ is taken first to calculate the strain, damage and stress fields.
 - The DIC-FE state fields are interpolated and smoothed as a basis for the crack detection step. The crack detection algorithm is employed to detect the crack path $\mathcal{C}(F_\gamma)$ at the near-failure level of load. The algorithm delivers the crack tip positions.
 - The obtained crack path is then refined by analyzing the cracking process in reverse order $F = [F_\gamma, 0]$. The difference between damage fields $\omega_{MN}(F_\gamma)$ and $\omega_{MN}(F), n = [0, n_C-1]$  obtained using DIC-FE-MDM calculation for each $U_{IJa}$ state are analyzed to refine the crack geometry in the previous states of loading. After that, the load range between the crack detection state and the ultimate load is analyzed based on the known cracking state at the level $F\tilde{}$.
 - Define a spline function representing the damage along the crack
 
## Current strategy

The crack detection load level $\gamma$ is used to identify the primary cracks. The cracks that develop at the level $\gamma = 0.9$ up to the 0.4 of the cross sectional height are recognized as primary cracks. Partially, secondary cracks and noise in DIC data and FE approximation are avoided  by smoothing the damage field within a radius that reflects the element resolution / characteristic length? Currently, the primary cracks are identified first to obtain the crack zones.

# Stress profile evaluation

 - M, N, Q - evaluated from the constitutive laws along the ligament
 - Crack-tip stress / strain / opening. 
 - How to include the history of damage into the stress evaluation (softening law)


## Constitutive laws along the ligament

The transition between strain concentration and discrete crack representation is based on the threshold value of the critical strain $\varepsilon_\mathrm{cr}$. This value corresponds to the strain with the onset of softening, i.e.
$$
 \varepsilon_\mathrm{cr} = \mathrm{argmax}_\varepsilon \mathcal{S}(\varepsilon).
$$
In case of a linear ascending branch, this relation is obtained as
$$
 \varepsilon_\mathrm{cr} = \frac{ f_\mathrm{t} }{ E }
$$

The stress in the ascending range is controlled by strain 
$$
\sigma_a
  = \left\{  
 \begin{array}{l}
 \mathcal{S}(\varepsilon)\; \mathrm{if} \; \varepsilon < \varepsilon_\mathrm{cr}
 \\
 \mathcal{S}(w) \; \mathrm{otherwise}
 \end{array}
 \right.
$$

An alternative approach is to introduce an equivalent strain or yield function.

# Todo

 - Add the extrapolation to include the top layer of the cross section [RC-done]
 - Separate the constitutive law for compression and tension [RC-done]
 - Include compression law considering the 3D stress state (plain stress)
 - In stress and force plot - align the ligament axis and show the capacity value for the reinforcement to indicate that it is yielding
 - bmcs_utils - improve the dependency management so that the tree nodes are not needed everywhere
 - Animation of the center of rotation - start with a localized crack up to 1/3 of the cross section

# Questions

- How to explain that the opening and sliding profiles are almost constant for shear crack while there is a center of rotation near the tip?

In [1]:
%matplotlib widget
import matplotlib.pylab as plt
from bmcs_shear.dic_crack import\
    DICGrid, DICStateFields, DICCrackList, DICCrack, DICCrackCOR
from bmcs_shear.shear_crack.crack_path import get_T_Lab
import numpy as np
np.seterr(divide='ignore', invalid='ignore');

In [2]:
dic_grid = DICGrid(U_factor=100, dir_name='B1_TV1', t=0.1)

Exception occurred in traits notification handler for event object: TraitChangeEvent(object=<bmcs_shear.dic_crack.dic_grid.DICGrid object at 0x7fa194695ea0>, name='dir_name', old='<unnamed>', new='B1_TV1')
Traceback (most recent call last):
  File "/home/rch/miniconda3/envs/bmcs_env/lib/python3.9/site-packages/traits/observation/_trait_event_notifier.py", line 122, in __call__
    self.dispatcher(handler, event)
  File "/home/rch/miniconda3/envs/bmcs_env/lib/python3.9/site-packages/traits/observation/observe.py", line 27, in dispatch_same
    handler(event)
  File "/home/rch/PycharmProjects/bmcs_shear_zone/bmcs_shear/dic_crack/dic_grid.py", line 78, in reset_beam_design
    self.sz_bd = self.get_beam_design()
  File "/home/rch/PycharmProjects/bmcs_shear_zone/bmcs_shear/dic_crack/dic_grid.py", line 91, in get_beam_design
    sz_bd.trait_set(**{key: params[key] for key in ['H', 'B', 'L']})
  File "/home/rch/miniconda3/envs/bmcs_env/lib/python3.9/site-packages/traits/has_traits.py", line 

trait <traits.ctrait.CTrait object at 0x7fa135efba40>
<bmcs_shear.beam_design.beam_design.RCBeamDesign object at 0x7fa135e65d10> matrix <bmcs_utils.trait_types.either_type2.EitherType2 object at 0x7fa13a571070>
trait <traits.ctrait.CTrait object at 0x7fa135efbae0>
<bmcs_shear.beam_design.beam_design.RCBeamDesign object at 0x7fa135e65d10> cross_section_shape <bmcs_utils.trait_types.either_type2.EitherType2 object at 0x7fa13a5710a0>
trait <traits.ctrait.CTrait object at 0x7fa1396962c0>
<bmcs_shear.beam_design.beam_design.RCBeamDesign object at 0x7fa135e65d10> cross_section_layout <bmcs_utils.trait_types.instance_type.Instance object at 0x7fa13a5070a0>
trait <traits.ctrait.CTrait object at 0x7fa137a3ab80>
<bmcs_shear.dic_crack.dic_grid.DICGrid object at 0x7fa194695ea0> sz_bd <bmcs_utils.trait_types.instance_type.Instance object at 0x7fa14f5b22b0>


In [14]:
submodels = []
self= dic_grid
for name in self.get_tree_items():
    trait = self.trait(name)
    trait_type = trait.trait_type
    print('xtrait', trait_type)
    name_ = trait_type.get_name_(name)
    print('name', name_)
    trait_ = getattr(self, name_, None)
    print('trait', trait_)
    if trait_ is None:
        raise ValueError('trait %s not found in %s' % (name_, self))
    submodels.append(trait_)


xtrait <bmcs_utils.trait_types.instance_type.Instance object at 0x7fc1cdc16160>
name sz_bd
trait None


ValueError: trait sz_bd not found in <bmcs_shear.dic_crack.dic_grid.DICGrid object at 0x7fc1984b2b80>

In [17]:
getattr(dic_grid, 'sz_bd')

AttributeError: 'RCBeamDesign' object has no attribute 'cross_section_shape_'

In [ ]:
dic_grid.interact()

In [25]:
l_cb = dic_grid.d_x * 1
dsf = DICStateFields(dic_grid=dic_grid)
dsf.tmodel_.trait_set(E=5000, c_T=0, nu=0.18, epsilon_0=0.0005, epsilon_f=0.02/l_cb, eps_max=0.005);
dsf.tmodel_.G_f * dic_grid.d_x

0.07805377028082207

In [26]:
dsf.eval()
dsf.dic_grid.sz_bd.matrix_.L_cr = 30

# Detect the cracks

In [27]:
dcl = DICCrackList(dsf=dsf, eta=1)

In [28]:
dcl.identify_cracks()
dcl.interact()

In [29]:
dcl['0'].x_t_unc_Ka

array([[ 67.99271262, 236.87615739],
       [ 67.99271262, 240.35227991],
       [ 67.99271262, 243.82840244],
       [ 67.99271262, 247.30452497],
       [ 67.99271262, 250.78064749],
       [ 67.99271262, 254.25677002],
       [ 67.99271262, 257.73289254],
       [ 67.99271262, 261.20901507],
       [ 67.99271262, 264.6851376 ],
       [ 67.99271262, 268.16126012],
       [ 67.99271262, 271.63738265],
       [ 67.99271262, 275.11350517],
       [ 67.99271262, 278.5896277 ],
       [ 67.99271262, 282.06575023],
       [ 67.99271262, 285.54187275],
       [ 67.99271262, 289.01799528],
       [ 67.99271262, 292.4941178 ],
       [ 67.99271262, 295.97024033],
       [ 67.99271262, 299.44636286],
       [ 67.99271262, 302.92248538],
       [ 67.99271262, 306.39860791],
       [ 67.99271262, 309.87473043],
       [ 67.99271262, 313.35085296]])

In [30]:
cl1 = dcl.items['1']
cl1.dic_grid.sz_bd.matrix_

# Analyze one crack

In [8]:
cl1.interact()

# Verify the center of rotation

In [9]:
cl1.cor.trait_set(step_N_COR=2)
cl1.cor.interact()

## Reverse crack detection - Put into separate notebook with explanation

 * Get the crack detection field and the primary cracks
   detected at the state of the mature cracking stage, 
   i.e. near the ultimate load.

### How to access the values of crack detection field at the crack tip

In [8]:
def get_crack_t(eta = 0.95):
    '''Get the crack history starting with the critical
    '''
    # identify the near feilure crack pattern
    t_eta_idx = dcl.dsf.dic_grid.get_F_eta_dic_idx(eta)
    dcl.dsf.dic_grid.end_t = t_eta_idx
    xx_NC, yy_NC, N_tip_C, M_NC = dcl.primary_cracks
    # pripare the list of crack tip states as a funciton of t
    N_tip_Ct = np.zeros((len(N_tip_C), t_eta_idx+1), np.int_)
    M_tip_Ct = np.zeros_like(N_tip_Ct)
    print('t_idx: ', end='')
    for t_idx in range(t_eta_idx, -1, -1):
        N_tip_C = np.copy(N_tip_C)
        N_tip_Ct[:, t_idx] = np.copy(N_tip_C)
        M_tip_Ct[:, t_idx] = M_NC[N_tip_C, np.arange(len(N_tip_C))]
        print(t_idx, end=' ')
        # get the crack detection field for the current time index
        dcl.dsf.dic_grid.end_t = t_idx
        _, _, cdf_MN = dcl.dsf.crack_detection_field
        # for each crack get the indexes starting from the current tip
        N_tip_CN = [np.arange(N_tip, -1, -1) for N_tip in N_tip_C]
        M_tip_CN = [M_NC[N_tip_t, C] for C, N_tip_t in enumerate(N_tip_CN)]
        # evaluate crack detection field along each crack
        cdf_CN = [cdf_MN[M_tip_N, N_tip_N] 
                  for M_tip_N, N_tip_N in zip(M_tip_CN, N_tip_CN)]
        # if cdf is zero completely - the crack did not emerge yet
        cdf_nonzero_C = np.array([np.sum(cdf_N) for cdf_N in cdf_CN]) > 0
        # find the row index distance from the tip with damage larger 
        # than threshold
        dN_C = np.array([np.argmax(cdf_C >= 0.2) for cdf_C in cdf_CN])
        N_tip_C[cdf_nonzero_C] -= dN_C[cdf_nonzero_C]
        N_tip_C[np.logical_not(cdf_nonzero_C)] = 0
    return M_tip_Ct, N_tip_Ct

In [9]:
get_crack_t()

AttributeError: 'DICGrid' object has no attribute 'get_F_eta_dic_idx'

In [ ]:
dcl.cracks_t

t_idx: 18 17 16 15 14 13 12 11 10 9 8 7 6 5 4 3 2 1 0 

([array([ 6, 13, 26, 38, 56, 74, 92]),
  array([ 6, 13, 26, 38, 56, 74, 92]),
  array([ 6, 13, 26, 38, 56, 74, 92]),
  array([ 6, 13, 26, 38, 56, 74, 92]),
  array([ 6, 13, 26, 38, 56, 74, 94]),
  array([ 6, 13, 27, 38, 56, 74, 95]),
  array([ 7, 13, 27, 38, 57, 74, 95]),
  array([ 8, 13, 27, 39, 58, 74, 95]),
  array([ 9, 13, 27, 39, 59, 74, 95]),
  array([ 9, 13, 27, 39, 60, 74, 95]),
  array([10, 13, 27, 39, 60, 75, 95]),
  array([10, 13, 27, 40, 60, 76, 95]),
  array([10, 14, 28, 40, 60, 76, 95]),
  array([10, 14, 28, 40, 60, 76, 95]),
  array([10, 15, 29, 41, 61, 76, 95]),
  array([10, 16, 29, 42, 61, 76, 95]),
  array([10, 16, 29, 42, 61, 76, 95]),
  array([10, 16, 29, 42, 61, 76, 95]),
  array([10, 16, 29, 42, 61, 76, 95])],
 [array([19, 18, 17, 19, 19, 16, 10]),
  array([19, 18, 17, 19, 19, 16, 10]),
  array([19, 18, 17, 19, 19, 16, 10]),
  array([19, 18, 17, 19, 19, 16, 10]),
  array([19, 18, 17, 19, 19, 16,  8]),
  array([18, 18, 16, 19, 19, 16,  7]),
  array([16, 18, 16, 19,

In [ ]:
_, _, cdf_eta_MN = dcl.dsf.crack_detection_field

(116, 28)

In [ ]:
xx_NC, yy_NC, N_tip_C, M_NC = dcl.primary_cracks

In [ ]:
N_tip_C

array([19, 18, 17, 19, 19, 16, 10])

In [ ]:
M_tip_C = M_NC[N_tip_C,np.arange(len(N_tip_C))]

In [ ]:
M_tip_C

array([ 6, 13, 26, 38, 56, 74, 92])

In [ ]:
cdf_eta_MN[M_tip_C, N_tip_C]

array([0.24249924, 0.27093637, 0.22398292, 0.38911079, 0.23217991,
       0.24228758, 0.2987822 ])

In [ ]:
_, _, cdf_MN = dcl.dsf.crack_detection_field
cdf_MN.shape

(116, 28)

In [ ]:
cdf_MN[M_tip_C, N_tip_C]

array([0.2102562 , 0.33028572, 0.22957975, 0.28097628, 0.31755808,
       0.30542646, 0.2718454 ])

### How to access the crack detection field for lower load levels

In [ ]:
dcl.dsf.dic_grid.end_t

11

In [ ]:
sp = dcl.items[3]

In [ ]:
sp.X_neutral_a

array([423.21144987, 235.67818428])

In [ ]:
x_00 = sp.dic_crack.C_cubic_spline(sp.z_N)

In [ ]:
x_00

array([472.76657979])

In [ ]:
sp.M

nan

In [ ]:
dcl.interact()

In [ ]:
crack_0 = dcl.items[2]

In [ ]:
crack_0.pos_F_y

(nan, nan)

In [ ]:
fig, ax = plt.subplots(1,1)
crack_0.dic_crack.plot_u_Nib(ax)
crack_0.dic_crack.plot_x_Na(ax)
crack_0.plot_COR(ax)
crack_0.plot_cor_markers(ax)
ax.axis('equal')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(248.36770344360858, 480.1367773488707, -7.550799748321647, 274.79547068760746)

In [ ]:
crack_0.M_N[20]

In [ ]:
dc_cor = DICCrackCOR(dic_crack=dcl.items[2])

In [ ]:
dc_cor.M_N.dtype

In [ ]:
slice_M_N = slice(dc_cor.M_N,dc_cor.M_N+3)

In [ ]:
X_mNa = np.array([dc_cor.X_MNa[dc_cor.M_N+i, np.arange(len(dc_cor.M_N))] for i in range(5)])

In [ ]:
X_mNa[-1,0]

In [ ]:
dc_cor.M0

In [ ]:
dc_cor.interact()

In [ ]:
dc_cor.X0_a - dc_cor.U0_a

In [ ]:
dc_cor.x_ref_MNa_scaled[dc_cor.M0, :dc_cor.N0_max]

array([[330.87272231,   4.85905338],
       [330.87254552,  13.86303457],
       [330.87358701,  22.868741  ],
       [330.87234085,  31.88218863],
       [330.87220829,  40.89703656]])